In [7]:
!pip install tensorflow-gpu
!pip install --no-deps tensorflow-io

  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [18 lines of output]
      Traceback (most recent call last):
        File "<string>", line 2, in <module>
        File "<pip-setuptools-caller>", line 34, in <module>
        File "/tmp/pip-install-delfyue0/tensorflow-gpu_c862fb26a5464431bb30d9802ae6f846/setup.py", line 37, in <module>
          raise Exception(TF_REMOVAL_WARNING)
      Exception:
      
      The "tensorflow-gpu" package has been removed!
      
      Please install "tensorflow" instead.
      
      Other than the name, the two packages have been identical
      since TensorFlow 2.1, or roughly since Sep 2019. For more
      information, see: pypi.org/project/tensorflow-gpu
      
      
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for tensorflow-gpu
  Running

In [9]:
import os
import tensorflow as tf
import numpy as np
from tqdm import tqdm

In [23]:
input_dir = "/kaggle/input/cocovqa/train2014"
!mkdir -p /kaggle/working/VQA_preprocessed/ImagesFeatures
output_dir = "/kaggle/working/VQA_preprocessed/ImagesFeatures"

In [24]:
def load_image(image_path):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, (224, 224))
    img = tf.keras.applications.vgg19.preprocess_input(img)
    return img, image_path

In [25]:
def feature_path(p, train_flag):
    # ./data/./img/train2014/COCO_train2014_000000178619.jpg
    directory = output_dir
    if not os.path.exists(directory):
        os.mkdir(directory)
    name = p.split('/')[-1].split('.')[0] + '.npy'
    if not os.path.exists(directory):
        os.mkdir(directory)
    directory = os.path.join(directory, 'train' if train_flag else 'test')
    if not os.path.exists(directory):
        os.mkdir(directory)
    return os.path.join(directory, name)

In [26]:
def extract_feature(img_path,
                    train_flag,
                    image_features_extract_model
                    ):
    # feature is in shape of [7*7, 512]

    unique_img = list(set(img_path))
    print("total image# to preprocess: ", len(unique_img))

    image_dataset = tf.data.Dataset.from_tensor_slices(unique_img)
    image_dataset = image_dataset.map(
        load_image, num_parallel_calls=tf.data.experimental.AUTOTUNE).batch(64)

    for img, path in tqdm(image_dataset):
        batch_features = image_features_extract_model(img)
        batch_features = tf.reshape(batch_features,
                                    (batch_features.shape[0],
                                     -1, batch_features.shape[3]))
        for f, p in zip(batch_features, path):
            path_of_feature = p.numpy().decode("utf-8")
            path_of_feature = feature_path(path_of_feature, train_flag=train_flag)
            np.save(path_of_feature, f.numpy())

In [27]:
trains = np.load('/kaggle/input/vqa-inputs/train.npy', allow_pickle=True).tolist()
print(len(trains))
for item in trains:
  path = item[0]
  name = path.split('/')[-1]
  path = os.path.join(input_dir,'train2014', name)
  print(path)
  break

443757
/kaggle/input/cocovqa/train2014/train2014/COCO_train2014_000000458752.jpg


In [28]:
trains = np.load('/kaggle/input/vqa-inputs/train.npy', allow_pickle=True).tolist()
img_paths = []
for item in trains:
  path = item[0]
  name = path.split('/')[-1]
  path = os.path.join(input_dir, '', name)
  img_paths.append(path)
  # using vgg19 pool5 to extract image feature

image_model = tf.keras.applications.VGG19(include_top=False,
                                          weights='imagenet')
new_input = image_model.input
hidden_layer = image_model.layers[-1].output
image_features_extract_model = tf.keras.Model(new_input, hidden_layer)
print("image model ready")

# extract image features
# Train

extract_feature(img_paths,
                True,
                image_features_extract_model
                )
print("train image done.")

image model ready
total image# to preprocess:  82783


100%|██████████| 1294/1294 [15:21<00:00,  1.40it/s]

train image done.


In [29]:
for item in trains[:10]:
  path = item[0]
  name = path.split('/')[-1][:-3] + 'npy'
  #org_path = os.path.join(input_dir, 'train2014', name)
  feature_path = os.path.join(output_dir, 'train', name)
  x = np.load(feature_path, allow_pickle=True).shape
  print(x)

  if not os.path.exists(feature_path):
    print('error')

(49, 512)
(49, 512)
(49, 512)
(49, 512)
(49, 512)
(49, 512)
(49, 512)
(49, 512)
(49, 512)
(49, 512)


In [30]:
import os
import re
import time
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [31]:
class vocab:

  def __init__(self, vocab_file):
    self.vocab = self.load_vocab(vocab_file)
    self.vocab2idx = {word: idx for idx, word in enumerate(self.vocab)}
    self.vocab_size = len(self.vocab)

  def load_vocab(self, vocab_file):
    with open(vocab_file, 'r') as f:
      vocab = [line.strip() for line in f]
    return vocab

  def word2idx(self, word):
    if word in self.vocab2idx:
      return self.vocab2idx[word]
    else:
      return self.vocab2idx['']

  def idx2word(self, idx):
    return self.vocab[idx]

In [39]:
question_vocab_dir = '/kaggle/input/vqa-preprocessed/preprocessed/Questions/question_vocabs.txt'
question_vocab = vocab(question_vocab_dir)

answer_vocab_dir = '/kaggle/input/vqa-preprocessed/preprocessed/Annotations/annotation_vocabs.txt'
answer_vocab = vocab(answer_vocab_dir)

In [40]:
def load_features(features_path):
  return np.load(features_path, allow_pickle=True)

In [41]:
max_qu_length = 30

In [42]:
def tokenizer(sentence):
    regex = re.compile(r'(\W+)')
    tokens = regex.split(sentence.lower())
    tokens = [w.strip() for w in tokens if len(w.strip()) > 0]
    return tokens[:-1]

In [43]:
def load_question(question):
  qu_tokens = tokenizer(question)
  qu2idx = np.full(max_qu_length, question_vocab.word2idx(''))
  qu2idx[:len(qu_tokens)] = [question_vocab.word2idx(token) for token in qu_tokens]
  return qu2idx

In [44]:
def load_answer(answer):
  answer_idx = answer_vocab.word2idx(answer)
  answer = np.zeros(answer_vocab.vocab_size)
  answer[answer_idx] = 1
  return answer

In [45]:
def preprocess(features_path, question, answer):
  features_path = features_path.numpy().decode('utf-8')
  question = question.numpy().decode('utf-8')
  answer = answer.numpy().decode('utf-8')

  features = load_features(features_path)
  question_vector = load_question(question)
  answer_vector = load_answer(answer)

  return (features, question_vector, answer_vector)

In [46]:
def get_tensors_ready(x,y,z):
  x.set_shape((49,512))
  y.set_shape((30,))
  z.set_shape((1000,))
  return ((x,y),z)

In [62]:
INPUT_DIR='/kaggle/input/vqa-inputs'

In [63]:
def build_dataset(file_name):
  data_dir = os.path.join(INPUT_DIR, file_name)
  data = np.load(data_dir, allow_pickle=True).tolist()
  if 'train' in file_name:
    features_path = '/kaggle/working/VQA_preprocessed/ImagesFeatures/train/'
  elif 'val' in file_name:
    features_path = '/kaggle/working/VQA_preprocessed/ImagesFeatures/test/'
  features = []
  questions = []
  answers = []
  for element in data:
    features.append(os.path.join(features_path, element[0][:-3] + 'npy'))
    questions.append(element[1])
    answers.append(element[2])

  dataset = tf.data.Dataset.from_tensor_slices((features, questions, answers))
  BATCH_SIZE = 128
  dataset = dataset.cache()
  dataset = dataset.map(lambda x, y, z: tf.py_function(func=preprocess,inp=[x, y, z], Tout=(tf.float32,tf.int32,tf.int32)), num_parallel_calls=tf.data.AUTOTUNE)
  dataset = dataset.map(get_tensors_ready)
  dataset = dataset.batch(BATCH_SIZE)
  dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
  return dataset

In [48]:
embedding_dict = {}
average_vec = np.zeros(300, "float32")
n = 0
with open("/kaggle/input/glove6b600d/glove.6B.300d.txt", 'r') as f:
  for line in f:
    values = line.split()
    word = values[0]
    vector = np.asarray(values[1: ], "float32")
    embedding_dict[word] = vector
    average_vec += vector
    n += 1

vocab_size = question_vocab.vocab_size
embedding_matrix = np.zeros((vocab_size, 300))

for i, word in enumerate(question_vocab.vocab):
  if i < vocab_size:
    emb_vector = embedding_dict.get(word)
    if emb_vector is not None:
      embedding_matrix[i] = emb_vector

average_vec = average_vec / n
embedding_matrix[1] = average_vec   #giving unkown words the average value of the words embeddings

In [59]:
#image model
im_input = tf.keras.layers.Input(shape=(49, 512))
x1 = tf.keras.layers.Flatten()(im_input)
x1 = tf.keras.layers.Dense(1024, activation='tanh')(x1)

#question model
vocab_size = question_vocab.vocab_size
q_input = tf.keras.layers.Input(shape=max_qu_length)
x2 = tf.keras.layers.Embedding(input_dim=vocab_size,
                               output_dim=300,
                               input_length=max_qu_length,
                               embeddings_initializer=tf.keras.initializers.Constant(embedding_matrix),
                               trainable=True)(q_input)
#x2 = tf.keras.layers.SimpleRNN(2048, return_sequences=True)(x2)
_, state_h, state_c = tf.keras.layers.LSTM(512, return_state=True)(x2)
x2 = tf.keras.layers.concatenate([state_h, state_c])

#combine features
out = tf.keras.layers.Multiply()([x1, x2])

#model output
num_answers = answer_vocab.vocab_size
out = tf.keras.layers.Dense(num_answers, activation='tanh')(out)
out = tf.keras.layers.Dropout(0.5)(out)
out = tf.keras.layers.Dense(num_answers, activation='tanh')(out)
out = tf.keras.layers.Dropout(0.5)(out)
out = tf.keras.layers.Dense(num_answers, activation='softmax')(out)

#model specs
model = tf.keras.Model(inputs=[im_input, q_input], outputs=[out])
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
               metrics=['accuracy'])


In [66]:
!mkdir '/kaggle/working/checkpoints_features'
model_checkpoint_path = "/kaggle/working/checkpoints_features/checkpoint.h5"
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(model_checkpoint_path)

training_history_path = "/kaggle/working/checkpoints_features/history.log"
history_callback = tf.keras.callbacks.CSVLogger(training_history_path, append=True)

mkdir: cannot create directory ‘/kaggle/working/checkpoints_features’: File exists


In [ ]:
train_dataset = build_dataset('train.npy')
#val_dataset = build_dataset('val.npy')
history = model.fit(train_dataset,
                    epochs=50,
                    shuffle=True,
                    #validation_data = train_dataset,
                    
                    callbacks = [model_checkpoint, history_callback])

Epoch 1/50
3467/3467 [==============================] - 864s 249ms/step - loss: 2.1173 - accuracy: 0.3844
Epoch 2/50
3467/3467 [==============================] - 870s 251ms/step - loss: 1.9408 - accuracy: 0.4053
Epoch 3/50
3467/3467 [==============================] - 877s 253ms/step - loss: 1.8561 - accuracy: 0.4200
Epoch 4/50
3467/3467 [==============================] - 870s 251ms/step - loss: 1.7973 - accuracy: 0.4302
Epoch 5/50
3467/3467 [==============================] - 871s 251ms/step - loss: 1.7504 - accuracy: 0.4379
Epoch 6/50
3467/3467 [==============================] - 878s 253ms/step - loss: 1.7050 - accuracy: 0.4483
Epoch 7/50
3467/3467 [==============================] - 880s 254ms/step - loss: 1.6598 - accuracy: 0.4598
Epoch 8/50
3467/3467 [==============================] - 873s 252ms/step - loss: 1.6169 - accuracy: 0.4710
Epoch 9/50
3467/3467 [==============================] - 877s 253ms/step - loss: 1.5765 - accuracy: 0.4833
Epoch 10/50
3467/3467 [=======================

In [ ]:
data = np.load('/content/drive/MyDrive/VQA_preprocessed/train.npy', allow_pickle=True).tolist()

In [ ]:

img_name = data[10][0]
question = data[10][1]
ans = data[10][2]

In [ ]:

print(question, ans)

In [ ]:
question = load_question(question)
ans = load_answer(ans)
img = np.load(os.path.join('/content/Features/train', img_name[:-3] + 'npy'), allow_pickle=True)

In [ ]:
img =  tf.expand_dims(img, axis=0)
question = tf.expand_dims(question, axis=0)
ans = tf.argmax(model.predict((img , question)), axis = 1)

In [ ]:
print(answer_vocab.idx2word(ans[0]))